In [ ]:
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from skimage.metrics import structural_similarity as ssim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
img=cv2.imread("/content/X-Ray_image.jpeg")
plt.imshow(img)

In [ ]:
channels=img.shape[2]

In [ ]:
img.shape

In [ ]:
channels

In [ ]:
if(channels==1): # If grayscale
  In=np.asarray(img.flat)
  In
if(channels==3): # If 3-channel rgb
  Lab_img=cv2.cvtColor(img,cv2.COLOR_BGR2LAB)
  L,a,b=cv2.split(Lab_img)
  L1=L.copy()
  In=np.asarray(L.flat)
  In

In [ ]:
plt.imshow(L,cmap='gray')

In [ ]:
L1

In [ ]:
#generating a histogram
def gen_histogram(image,bins):
  hist=np.zeros(bins)

  for i in image:
    hist[i]+=1

  return hist

In [ ]:
Histogram=gen_histogram(In,256)
plt.plot(Histogram)

In [ ]:
Histogram

In [ ]:
N=np.sum(Histogram)
N

def calc_pdf(hist): #calculating probability density function
  pdf=np.zeros(256)
  for i in range(0,256):
    pdf[i]=hist[i]/N

  return pdf

In [ ]:
PDF=calc_pdf(Histogram)
PDF

In [ ]:
def new_intensities(image):
  for i in range(0,256):
    image[i]=image[i]*255
  return image


def calc_cdf(image): #Calculating cumulative density function
  cdf=np.zeros(256)
  for i in range(0,256):
    if i==256:
       cdf[i]= np.sum(image)
    else:
      for j in range(0,i+1):
        cdf[i]+=image[j]

  return cdf

In [ ]:
CDF=calc_cdf(PDF)
CDF

In [ ]:
New_pixel=new_intensities(CDF)
New_pixel

In [ ]:
New_pixel_int=New_pixel.astype('uint8')
New_pixel_int

In [ ]:
def get_new_image(New_values, Intensity_array):
    for i in range(len(Intensity_array)):
        if 0 <= Intensity_array[i] < len(New_values):
            Intensity_array[i] = New_values[Intensity_array[i]]
    return Intensity_array

In [ ]:
Equal_L=get_new_image(New_pixel_int, In)
Equal_L

In [ ]:
Equal_L.shape

In [ ]:
Equal_Hist=gen_histogram(Equal_L,256)
plt.plot(Equal_Hist)

In [ ]:
Equal_L = np.reshape(Equal_L, L.shape)

In [ ]:
plt.imshow(L1, cmap='gray')

In [ ]:
L1

In [ ]:
plt.imshow(Equal_L, cmap='gray')

In [ ]:
Equal_L

In [ ]:
score, _ = ssim(L1, Equal_L, full=True)

In [ ]:
score

In [ ]:
_

In [ ]:
def evaluate_fitness(original_image, processed_image):

    ogn_flat = original_image.flatten()
    pcs_flat = processed_image.flatten()

    mse = np.mean((ogn_flat - pcs_flat) ** 2)
    max_pixel = 255.0

    psnr = 20 * np.log10(max_pixel / np.sqrt(mse))
    return psnr

In [ ]:
PSNR=evaluate_fitness(L1, Equal_L)

In [ ]:
PSNR

In [ ]:
New_img= cv2.merge((Equal_L,a,b))

In [ ]:
plt.imshow(New_img)

In [ ]:
Hist_img= cv2.cvtColor(New_img,cv2.COLOR_LAB2BGR)

In [ ]:
plt.imshow(Hist_img)

In [ ]:
import matplotlib.pyplot as plt
import cv2

# Convert images from BGR to RGB format (assuming OpenCV reads images in BGR format)
original_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
enhanced_image = cv2.cvtColor(Hist_img, cv2.COLOR_BGR2RGB)

# Create a figure and axis objects
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Display original image
axes[0].imshow(original_image)
axes[0].set_title('Original')  # Add label 'Before' to the original image
axes[0].axis('off')

# Display enhanced image
axes[1].imshow(enhanced_image)
axes[1].set_title('Enhanced')  # Add label 'After' to the enhanced image
axes[1].axis('off')

# Show the plot
plt.show()


In [ ]:
L